<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#MNIST-Dataset" data-toc-modified-id="MNIST-Dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>MNIST Dataset</a></span><ul class="toc-item"><li><span><a href="#Get-the-MNIST-data" data-toc-modified-id="Get-the-MNIST-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get the MNIST data</a></span></li><li><span><a href="#MLP-in-TensorFlow" data-toc-modified-id="MLP-in-TensorFlow-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>MLP in TensorFlow</a></span></li><li><span><a href="#MLP-in-Keras" data-toc-modified-id="MLP-in-Keras-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>MLP in Keras</a></span></li><li><span><a href="#MLP-in-TFLearn" data-toc-modified-id="MLP-in-TFLearn-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>MLP in TFLearn</a></span></li></ul></li><li><span><a href="#TimeSeries-Data---MLP---Keras" data-toc-modified-id="TimeSeries-Data---MLP---Keras-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>TimeSeries Data - MLP - Keras</a></span><ul class="toc-item"><li><span><a href="#Prepare-the-data" data-toc-modified-id="Prepare-the-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Prepare the data</a></span></li><li><span><a href="#Build,-Train-and-Evaluate-the-Model" data-toc-modified-id="Build,-Train-and-Evaluate-the-Model-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Build, Train and Evaluate the Model</a></span></li></ul></li></ul></div>

# MultiLayer Perceptron <a class="tocSkip">

In [1]:
import numpy as np
np.random.seed(123)
print("NumPy:{}".format(np.__version__))

import pandas as pd
print("Pandas:{}".format(pd.__version__))

import math
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 10
print("Matplotlib:{}".format(mpl.__version__))

import tensorflow as tf
tf.set_random_seed(123)
print("TensorFlow:{}".format(tf.__version__))

import keras
print("Keras:{}".format(keras.__version__))

NumPy:1.14.5
Pandas:0.22.0
Matplotlib:2.2.2
TensorFlow:1.11.0
Keras:2.2.4


Using TensorFlow backend.


In [2]:
DATASETSLIB_HOME = '../datasetslib'
import sys
if not DATASETSLIB_HOME in sys.path:
    sys.path.append(DATASETSLIB_HOME)
%reload_ext autoreload
%autoreload 2
import datasetslib

from datasetslib import util as dsu

In [3]:
datasetslib.datasets_root = os.path.join(os.path.expanduser('~'),'datasets')

# MNIST Dataset

## Get the MNIST data

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(os.path.join(datasetslib.datasets_root, 'mnist'),
                                  one_hot=True)

X_train = mnist.train.images
X_test = mnist.test.images
Y_train = mnist.train.labels
Y_test = mnist.test.labels

num_outputs = 10  # 0-9 digits
num_inputs = 784  # total pixels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/ubuntu/datasets/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/ubuntu/datasets/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/ubuntu/datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/ubuntu/datasets/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## MLP in TensorFlow

In [5]:
tf.reset_default_graph()

In [6]:
def mlp(x, num_inputs, num_outputs, num_layers, num_neurons):
    w = []
    b = []
    for i in range(num_layers):
        # weights
        w.append(tf.Variable(tf.random_normal(
            [num_inputs if i == 0 else num_neurons[i - 1],
             num_neurons[i]]),
            name="w_{0:04d}".format(i)
        ))
        # biases
        b.append(tf.Variable(tf.random_normal(
            [num_neurons[i]]),
            name="b_{0:04d}".format(i)
        ))
    w.append(tf.Variable(tf.random_normal(
        [num_neurons[num_layers - 1] if num_layers > 0 else num_inputs,
         num_outputs]), name="w_out"))
    b.append(tf.Variable(tf.random_normal([num_outputs]), name="b_out"))

    # x is input layer
    layer = x
    # add hidden layers
    for i in range(num_layers):
        layer = tf.nn.relu(tf.matmul(layer, w[i]) + b[i])
    # add output layer
    layer = tf.matmul(layer, w[num_layers]) + b[num_layers]

    return layer


def mnist_batch_func(batch_size=100):
    X_batch, Y_batch = mnist.train.next_batch(batch_size)
    return [X_batch, Y_batch]


def tensorflow_classification(n_epochs, n_batches,
                              batch_size, batch_func,
                              model, optimizer, loss, accuracy_function,
                              X_test, Y_test):
    with tf.Session() as tfs:
        tfs.run(tf.global_variables_initializer())
        for epoch in range(n_epochs):
            epoch_loss = 0.0
            for batch in range(n_batches):
                X_batch, Y_batch = batch_func(batch_size)
                feed_dict = {x: X_batch, y: Y_batch}
                _, batch_loss = tfs.run([optimizer, loss], feed_dict)
                epoch_loss += batch_loss
            average_loss = epoch_loss / n_batches
            print("epoch: {0:04d}   loss = {1:0.6f}".format(
                epoch, average_loss))
        feed_dict = {x: X_test, y: Y_test}
        accuracy_score = tfs.run(accuracy_function, feed_dict=feed_dict)
        print("accuracy={0:.8f}".format(accuracy_score))

In [7]:
tf.reset_default_graph()

In [8]:
num_layers = 0
num_neurons = []
learning_rate = 0.01
n_epochs = 50
batch_size = 100
n_batches = int(mnist.train.num_examples / batch_size)

# input images
x = tf.placeholder(dtype=tf.float32, name="x", shape=[None, num_inputs])
# target output
y = tf.placeholder(dtype=tf.float32, name="y", shape=[None, num_outputs])

model = mlp(x=x,
            num_inputs=num_inputs,
            num_outputs=num_outputs,
            num_layers=num_layers,
            num_neurons=num_neurons)

# loss function
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
# optimizer function
optimizer = tf.train.GradientDescentOptimizer(
    learning_rate=learning_rate).minimize(loss)

predictions_check = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))

tensorflow_classification(n_epochs=n_epochs,
                          n_batches=n_batches,
                          batch_size=batch_size,
                          batch_func=mnist_batch_func,
                          model=model,
                          optimizer=optimizer,
                          loss=loss,
                          accuracy_function=accuracy_function,
                          X_test=mnist.test.images,
                          Y_test=mnist.test.labels
                          )

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

epoch: 0000   loss = 7.310428
epoch: 0001   loss = 4.130082
epoch: 0002   loss = 2.953550
epoch: 0003   loss = 2.362319
epoch: 0004   loss = 2.018068
epoch: 0005   loss = 1.792703
epoch: 0006   loss = 1.632227
epoch: 0007   loss = 1.511098
epoch: 0008   loss = 1.415811
epoch: 0009   loss = 1.338710
epoch: 0010   loss = 1.274726
epoch: 0011   loss = 1.220370
epoch: 0012   loss = 1.173759
epoch: 0013   loss = 1.132906
epoch: 0014   loss = 1.096913
epoch: 0015   loss = 1.064849
epoch: 0016   loss = 1.035915
epoch: 0017   loss = 1.009694
epoch: 0018   loss = 0.985728
epoch: 0019   loss = 0.963920
epoch: 0020   loss = 0.943645
epoch: 0021   loss = 0.924987
epoch: 0022   loss = 0.907552
epoch: 0023   loss = 0.891403
epoch: 0024   loss = 0.876160
epoch: 0025   loss = 0.861992
epoch: 0026   loss = 0.848590

In [9]:
tf.reset_default_graph()

In [10]:
num_layers = 1 
num_neurons = []
for i in range(num_layers):
    num_neurons.append(8)
    
learning_rate = 0.01
n_epochs = 50
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

# input images
x = tf.placeholder(dtype=tf.float32, name="x", shape=[None, num_inputs]) 
# target output
y = tf.placeholder(dtype=tf.float32, name="y", shape=[None, num_outputs]) 

model = mlp(x=x, 
            num_inputs=num_inputs, 
            num_outputs=num_outputs, 
            num_layers=num_layers, 
            num_neurons=num_neurons)

# loss function
#loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(model), axis=1))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
# optimizer function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
predictions_check = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))

tensorflow_classification(n_epochs=n_epochs, 
                          n_batches=n_batches, 
                          batch_size=batch_size, 
                          batch_func=mnist_batch_func, 
                          model = model, 
                          optimizer = optimizer, 
                          loss = loss, 
                          accuracy_function = accuracy_function, 
                          X_test = mnist.test.images, 
                          Y_test = mnist.test.labels
                         )

epoch: 0000   loss = 3.982879
epoch: 0001   loss = 2.290242
epoch: 0002   loss = 2.146824
epoch: 0003   loss = 2.049216
epoch: 0004   loss = 1.977785
epoch: 0005   loss = 1.922585
epoch: 0006   loss = 1.875601
epoch: 0007   loss = 1.832573
epoch: 0008   loss = 1.790754
epoch: 0009   loss = 1.748355
epoch: 0010   loss = 1.704254
epoch: 0011   loss = 1.657746
epoch: 0012   loss = 1.609882
epoch: 0013   loss = 1.561166
epoch: 0014   loss = 1.514168
epoch: 0015   loss = 1.469856
epoch: 0016   loss = 1.428373
epoch: 0017   loss = 1.388716
epoch: 0018   loss = 1.349924
epoch: 0019   loss = 1.313423
epoch: 0020   loss = 1.277759
epoch: 0021   loss = 1.244732
epoch: 0022   loss = 1.213374
epoch: 0023   loss = 1.186042
epoch: 0024   loss = 1.161652
epoch: 0025   loss = 1.138693
epoch: 0026   loss = 1.118391
epoch: 0027   loss = 1.099992
epoch: 0028   loss = 1.082146
epoch: 0029   loss = 1.066611
epoch: 0030   loss = 1.051637
epoch: 0031   loss = 1.036788
epoch: 0032   loss = 1.023856
epoch: 003

In [11]:
tf.reset_default_graph()

In [12]:
num_layers = 2
num_neurons = []
for i in range(num_layers):
    num_neurons.append(256)
    
learning_rate = 0.01
n_epochs = 50
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

# input images
x = tf.placeholder(dtype=tf.float32, name="x", shape=[None, num_inputs]) 
# target output
y = tf.placeholder(dtype=tf.float32, name="y", shape=[None, num_outputs]) 

model = mlp(x=x, 
            num_inputs=num_inputs, 
            num_outputs=num_outputs, 
            num_layers=num_layers, 
            num_neurons=num_neurons)

# loss function
#loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(model), axis=1))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, 
                                                              labels=y))
# optimizer function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

predictions_check = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))

tensorflow_classification(n_epochs=n_epochs, 
                          n_batches=n_batches, 
                          batch_size=batch_size, 
                          batch_func=mnist_batch_func, 
                          model = model, 
                          optimizer = optimizer, 
                          loss = loss, 
                          accuracy_function = accuracy_function, 
                          X_test = mnist.test.images, 
                          Y_test = mnist.test.labels
                         )

epoch: 0000   loss = 59.286007
epoch: 0001   loss = 14.451488
epoch: 0002   loss = 8.853264
epoch: 0003   loss = 6.265361
epoch: 0004   loss = 4.805574
epoch: 0005   loss = 3.763518
epoch: 0006   loss = 3.057002
epoch: 0007   loss = 2.483716
epoch: 0008   loss = 2.039015
epoch: 0009   loss = 1.686357
epoch: 0010   loss = 1.451302
epoch: 0011   loss = 1.231050
epoch: 0012   loss = 1.050846
epoch: 0013   loss = 0.843715
epoch: 0014   loss = 0.764786
epoch: 0015   loss = 0.648526
epoch: 0016   loss = 0.553345
epoch: 0017   loss = 0.484354
epoch: 0018   loss = 0.403943
epoch: 0019   loss = 0.348224
epoch: 0020   loss = 0.299267
epoch: 0021   loss = 0.250771
epoch: 0022   loss = 0.220450
epoch: 0023   loss = 0.197737
epoch: 0024   loss = 0.170108
epoch: 0025   loss = 0.129920
epoch: 0026   loss = 0.107821
epoch: 0027   loss = 0.101173
epoch: 0028   loss = 0.084048
epoch: 0029   loss = 0.067873
epoch: 0030   loss = 0.059228
epoch: 0031   loss = 0.050879
epoch: 0032   loss = 0.043111
epoch: 0

## MLP in Keras


In [13]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [14]:
tf.reset_default_graph()
keras.backend.clear_session()

In [15]:
num_layers = 2
num_neurons = []
for i in range(num_layers):
    num_neurons.append(256)
    
learning_rate = 0.01
n_epochs = 50
batch_size = 100

model = Sequential()
model.add(Dense(units=num_neurons[0], activation='relu', 
                input_shape=(num_inputs,)))
model.add(Dense(units=num_neurons[1], activation='relu'))
model.add(Dense(units=num_outputs, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=learning_rate),
              metrics=['accuracy'])

model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=n_epochs)

score = model.evaluate(X_test, Y_test)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
55000/55000 [==============================] - 2s 32us/step - loss: 1.0688 - acc: 0.7467
Epoch 2/50
55000/55000 [==============================] - 2s 29us/step - loss: 0.4358 - acc: 0.8813
Epoch 3/50
55000/55000 [==============================] - 2s 29us/step - loss: 0.3535 - acc: 0.9003
Epoch 4/50
55000/55000 [==============================] - 2s 29us/step - loss: 0.3147 - acc: 0.9103


## MLP in TFLearn

In [16]:
import tflearn

In [17]:
tf.reset_default_graph()

In [18]:
num_layers = 2
num_neurons = []
for i in range(num_layers):
    num_neurons.append(256)
    
learning_rate = 0.01
n_epochs = 50
batch_size = 100

# Build deep neural network
input_layer = tflearn.input_data(shape=[None, num_inputs])
dense1 = tflearn.fully_connected(input_layer, num_neurons[0], activation='relu')
dense2 = tflearn.fully_connected(dense1, num_neurons[1], activation='relu')
softmax = tflearn.fully_connected(dense2, num_outputs, activation='softmax')

optimizer = tflearn.SGD(learning_rate=learning_rate)
net = tflearn.regression(softmax, optimizer=optimizer, 
                         metric=tflearn.metrics.Accuracy(), 
                         loss='categorical_crossentropy')
model = tflearn.DNN(net)

model.fit(X_train, Y_train, 
          n_epoch=n_epochs, batch_size=batch_size, 
          show_metric=True, run_id='dense_model')

score = model.evaluate(X_test, Y_test)
print('Test accuracy:', score[0])

Training Step: 27499  | total loss: 0.10231 | time: 2.675s
| SGD | epoch: 050 | loss: 0.10231 - acc: 0.9742 -- iter: 54900/55000
Training Step: 27500  | total loss: 0.10065 | time: 2.680s
| SGD | epoch: 050 | loss: 0.10065 - acc: 0.9738 -- iter: 55000/55000
--
Test accuracy: 0.9653


# TimeSeries Data - MLP - Keras

## Prepare the data

In [19]:
import os

dataframe = pd.read_csv(os.path.join(datasetslib.datasets_root, 
                                     'ts-data', 
                                     'international-airline-passengers-cleaned.csv'), 
                        usecols=[1],header=0)
dataset = dataframe.values
dataset = dataset.astype('float32')

FileNotFoundError: File b'/home/ubuntu/datasets/ts-data/international-airline-passengers-cleaned.csv' does not exist

In [ ]:
# normalize the dataset
#scaler = skpp.MinMaxScaler(feature_range=(0, 1))
#normalized_dataset = scaler.fit_transform(dataset)

In [ ]:
# split into train and test sets
train,test=dsu.train_test_split(dataset,train_size=0.67)
print(len(train), len(test))

In [ ]:
# reshape into X=t-1,t and Y=t+1
n_x=2
n_y=1

X_train, Y_train, X_test, Y_test = dsu.mvts_to_xy(train,test,n_x=n_x,n_y=n_y)

## Build, Train and Evaluate the Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [ ]:
tf.reset_default_graph()
keras.backend.clear_session()

In [ ]:
num_layers = 2
num_neurons = [8,8]
n_epochs = 50
batch_size = 2

model = Sequential()
model.add(Dense(num_neurons[0], activation='relu', input_shape=(n_x,)))
model.add(Dense(num_neurons[1], activation='relu'))
model.add(Dense(units=1))
model.summary()

model.compile(loss='mse', optimizer='adam')

model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=n_epochs)

score = model.evaluate(X_test, Y_test)
print('\nTest mse:', score)
print('Test rmse:', math.sqrt(score))

In [ ]:
# make predictions
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

# shift train predictions for plotting
Y_train_pred_plot = np.empty_like(dataset)
Y_train_pred_plot[:, :] = np.nan
Y_train_pred_plot[n_x-1:len(Y_train_pred)+n_x-1, :] = Y_train_pred

# shift test predictions for plotting
Y_test_pred_plot = np.empty_like(dataset)
Y_test_pred_plot[:, :] = np.nan
Y_test_pred_plot[len(Y_train_pred)+(n_x*2)-1:len(dataset)-1, :] = Y_test_pred

# plot baseline and predictions
plt.plot(dataset,label='Original Data')
plt.plot(Y_train_pred_plot,label='Y_train_pred')
plt.plot(Y_test_pred_plot,label='Y_test_pred')
plt.legend()
plt.show()